# 实验 2.1 基于 Python 的 phe 库完成加法和标量乘法的验证

In [1]:
from phe import paillier # 开源库
import time # 做性能测试

##################### 设置参数
print("默认私钥大小：", paillier.DEFAULT_KEYSIZE)
#生成公私钥
public_key, private_key = paillier.generate_paillier_keypair()
# 测试需要加密的数据
message_list = [3.1415926, 100, - 4.6e - 12]

##################### 加密操作
time_start_enc = time.time()
encrypted_message_list = [public_key.encrypt(m) for m in message_list]
time_end_enc = time.time()
print("加密耗时s：", time_end_enc - time_start_enc)
print("加密数据（3.1415926）:", encrypted_message_list[0].ciphertext())

##################### 解密操作
time_start_dec = time.time()
decrypted_message_list = [private_key.decrypt(c) for c in encrypted_message_list]
time_end_dec = time.time()
print("解密耗时s：", time_end_dec-time_start_dec)
print("原始数据（3.1415926）:", decrypted_message_list[0])

##################### 测试加法和乘法同态
a,b,c = encrypted_message_list # a,b,c分别为对应密文
a_sum = a + 5 # 密文加明文，已经重载了+运算符
a_sub = a - 3 # 密文加明文的相反数，已经重载了-运算符
b_mul = b * 6 # 密文乘明文,数乘
c_div = c / - 10.0 # 密文乘明文的倒数
print("a + 5 密文: ",a.ciphertext()) # 密文纯文本形式
print("a + 5 = ",private_key.decrypt(a_sum))
print("a - 3 ",private_key.decrypt(a_sub))
print("b * 6 = ",private_key.decrypt(b_mul))
print("c / - 10.0 = ",private_key.decrypt(c_div))
##密文加密文
print((private_key.decrypt(a) + private_key.decrypt(b)) == private_key.decrypt(a + b))
#报错，不支持a*b，即两个密文直接相乘
#print((private_key.decrypt(a)+private_key.decrypt(b))==private_key.decrypt(a*b))

默认私钥大小： 3072
加密耗时s： 1.5536248683929443
加密数据（3.1415926）: 93134808166930738472489203846671101179293758656288961560775848854943318836406874406679484252571919197568208716840448523034754427719062709395396714338445304244385884302755379541985251184633819020807046201836219325676193215167839988179904974168366637414392174238681550030101227717329166817714918230595701395557774226987827101818573413739094512826827770445720032409582519380798708456008472924033381344073586726202939760839429521582655075578852444240416171365749340177131978668853235282101921319905297968921170442388658167088520449081772350164109219327214552581240084502296579501766259385099402380463428879637783677739735271848701847541017913190978973433200756579601026081297075672707158335301093512952431128014940235672891502406563080116874820294570528174289088834353792522069515115772279052567898481414568683997864775790899064277487512558385874249880256833721629708035035424544391726803835889238071055588767550923279604798486076338682765369532155

# 实验 2.2 基于 Python 的 phe 库完成隐私信息获取的功能：服务器端拥有多个数值，要求客户端能基于 Paillier 实现从服务器读取一个指定的数值并正确解密，但服务器不知道所读取的是哪一个

In [2]:
from phe import paillier # 开源库
import random # 选择随机数

##################### 设置参数
# 服务器端保存的数值
message_list = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
length = len(message_list)
# 客户端生成公私钥
public_key, private_key = paillier.generate_paillier_keypair()
# 客户端随机选择一个要读的位置
pos = random.randint(0, length - 1)
print("要读起的数值位置为：", pos)

##################### 客户端生成密文选择向量
select_list = []
enc_list = []
for i in range(length):
    select_list.append( i == pos )
    enc_list.append( public_key.encrypt(select_list[i]) )
# for element in select_list:
#     print(element)
# for element in enc_list:
#     print(private_key.decrypt(element))

##################### 服务器端进行运算
c = 0
for i in range(length):
    c = c + message_list[i] * enc_list[i]
print("产生密文：",c.ciphertext())

##################### 客户端进行解密
m = private_key.decrypt(c)
print("得到数值：", m)

要读起的数值位置为： 8
产生密文： 633136316356432197991993051460176355281915521250070153535344657491952080558296517009927088790885003527592464655381519185914860025559660016604744098604327226904248534184611374557385207338230155334113864257027383415095726589928782283886327074965983278193394936264039125129682361430545049332665830083874030998399694788147126836618047386286893131904002392600862713562667253529549368562254353289597923430550991447383925137619338504352216319757529932820966181784607689647485289678108470345406764176125532497711822081995089842689088520174775563969294153427847841319271240557063361282251747465896717800417782774077287609071664816109288533101083170969122800776526583533019224132343176263884711001042488691741928963103870314772045848460811830884132406878776309530746346250636831222813363220447804185990541031893290253840933631940016511180011106370642009092152854977028868250344523940841745072886767010592893007126540112914371116917624052796548741370491290426667829979550326354094424643899445